In [17]:
import torch
from torch import nn,optim
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms, models
from torchvision.datasets import ImageFolder

import matplotlib.pyplot as plt
import os,sys
from statistics import mean
import numpy as np

In [18]:
current_path = os.path.dirname(os.path.abspath(sys.argv[1]))
TRANSFORM = transforms.Compose([transforms.Resize(128), transforms.CenterCrop(127), transforms.ToTensor()])

train = ImageFolder(current_path + '/animal_pics/training_set', transform = TRANSFORM)
test = ImageFolder(current_path + '/animal_pics/test_set', transform = TRANSFORM)

In [19]:
# Loading pretrained net
net = models.vgg16(pretrained=True)
print(net)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [20]:
# setting last layer to have 2 output features
net.classifier[6] = nn.Linear(in_features=4096, out_features=2)

In [21]:
# getting params to update for the optimizer
params_to_update = []
update_params_name = ['classifier.6.weight','classifier.6.bias']

for name, param in net.named_parameters():
    if name in update_params_name :
        param.requires_grad = True
        params_to_update.append(param)
    else :
        param.requires_grad = False

In [22]:
def train_model(model, opt, criterion, epochs, train_loader, test_loader):

    stats_dict = {'all_loss':[], 'mean_loss':[], 'accuracy':[], 'models':[]}

    try : 
        for epoch in range(epochs):
        
            print(f'EPOCH {epoch+1}')
            print()
            
            batch_counter = 0
            mean_loss_for_epoch = []

            for x,y in train_loader :

                if batch_counter % 5 == 0 :
                    print(str(batch_counter) + ' --> ', end = '', flush = True)

                model.train()
                opt.zero_grad()

                yhat = model.forward(x)
                loss = criterion(yhat,y)

                stats_dict['all_loss'].append(loss.item())
                mean_loss_for_epoch.append(loss.item())

                loss.backward()
                opt.step()
                batch_counter += 1

            stats_dict['mean_loss'].append(mean(mean_loss_for_epoch))
            stats_dict['models'].append(model)
            print()
            print('Mean loss for epoch :', mean(mean_loss_for_epoch))

            correct = 0
            incorrect = 0
            for x,y in test_loader :

                model.eval()
                yhat = model.forward(x)

                for i in range(len(yhat)) : 
                    if torch.argmax(yhat[i]) == y[i] :
                        correct += 1
                    else :
                        incorrect += 1

            acc = (correct / (incorrect + correct)) * 100
            print('Accuracy on test set : ', acc, '%')
            stats_dict['accuracy'].append(acc)
            print('-' * 50)

        return stats_dict
    except :
        return stats_dict

In [23]:
BATCH_SIZE = 10
EPOCHS = 5
LR = 0.001
MOMENTUM = 0.9

train_loader = DataLoader(dataset = train, batch_size = BATCH_SIZE, shuffle = True)
test_loader = DataLoader(dataset = test, batch_size = 2000, shuffle = True)

opt = optim.SGD(params_to_update, lr = LR, momentum = MOMENTUM)
loss = nn.CrossEntropyLoss()

stats = train_model(model = net, opt = opt, criterion = loss, epochs = EPOCHS, train_loader = train_loader, test_loader = test_loader)

EPOCH 1

0 --> 5 --> 10 --> 15 --> 20 --> 25 --> 30 --> 35 --> 40 --> 45 --> 50 --> 55 --> 60 --> 65 --> 70 --> 75 --> 80 --> 85 --> 90 --> 95 --> 100 --> 105 --> 110 --> 115 --> 120 --> 125 --> 130 --> 135 --> 140 --> 145 --> 150 --> 155 --> 160 --> 165 --> 170 --> 175 --> 180 --> 185 --> 190 --> 195 --> 200 --> 205 --> 210 --> 215 --> 220 --> 225 --> 230 --> 235 --> 240 --> 245 --> 250 --> 255 --> 260 --> 265 --> 270 --> 275 --> 280 --> 285 --> 290 --> 295 --> 300 --> 305 --> 310 --> 315 --> 320 --> 325 --> 330 --> 335 --> 340 --> 345 --> 350 --> 355 --> 360 --> 365 --> 370 --> 375 --> 380 --> 385 --> 390 --> 395 --> 400 --> 405 --> 410 --> 415 --> 420 --> 425 --> 430 --> 435 --> 440 --> 445 --> 450 --> 455 --> 460 --> 465 --> 470 --> 475 --> 480 --> 485 --> 490 --> 495 --> 500 --> 505 --> 510 --> 515 --> 520 --> 525 --> 530 --> 535 --> 540 --> 545 --> 550 --> 555 --> 560 --> 565 --> 570 --> 575 --> 580 --> 585 --> 590 --> 595 --> 600 --> 605 --> 610 --> 615 --> 620 --> 625 --> 630 -